In [18]:
# labels should be on abother folder

In [38]:
import os
from glob import glob


In [41]:
name = "_multi_DTM_LiDAR"
name = "label"
tifpath = f"/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/*/*{name}*.tif"
xmlpath = f"/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/*/*{name}*.aux.xml"

In [43]:
path = xmlpath #tifpath
files = glob(path); print(len(files))
for fi in files:os.remove(fi)

28


In [28]:
exclude_suffixes = ["label_kmeans_slope.tif", "label_kmeans.tif", "label_landdata.tif", "label_slope_adathresh.tif"]

# Filtrando os arquivos
filtered_files = [f for f in files if not any(f.endswith(suffix) for suffix in exclude_suffixes)]

print(len(filtered_files))  

205


In [29]:
for fi in filtered_files:os.remove(fi)

In [45]:
dtm_path = "/media/ljp238/12TBWolf/RSPROX/GEDI_GRID/tiles/N13E103/N13E103_GEDI03_dtm____GWR_12.tif"
dsm_path = "/media/ljp238/12TBWolf/RSPROX/GEDI_GRID/tiles/N13E103/N13E103_GEDI03_dsm____GWR_12.tif"

outdir = "/home/ljp238/Desktop/testing_gedi/"

In [51]:
import rasterio
import numpy as np
import os

def raster_calc(dsm_path, dtm_path, operation, out_dir, output_name):
    """
    Realiza a soma ou subtração de dois rasters (DSM e DTM) e salva o resultado.

    :param dsm_path: Caminho do arquivo DSM (Digital Surface Model).
    :param dtm_path: Caminho do arquivo DTM (Digital Terrain Model).
    :param operation: Operação a ser realizada ('add' para soma, 'subtract' para subtração).
    :param out_dir: Diretório de saída para salvar o resultado.
    :param output_name: Nome do arquivo de saída (sem extensão).
    """

    # Abrir os arquivos raster
    with rasterio.open(dsm_path) as dsm, rasterio.open(dtm_path) as dtm:
        if dsm.shape != dtm.shape:
            raise ValueError("Os rasters DSM e DTM devem ter o mesmo tamanho.")

        dsm_data = dsm.read(1)  # Lê a primeira banda
        dtm_data = dtm.read(1)  # Lê a primeira banda

        # Realizar a operação desejada
        if operation == 'add':
            result_data = dsm_data + dtm_data
        elif operation == 'subtract':
            result_data = dsm_data - dtm_data
        else:
            raise ValueError("Operação inválida. Escolha 'add' ou 'subtract'.")

        # Configuração dos metadados para o arquivo de saída
        output_meta = dsm.meta.copy()
        output_meta.update({"dtype": "float32"})  # Garante compatibilidade

        # Criar diretório de saída se não existir
        
        os.makedirs(out_dir, exist_ok=True)
        output_path = os.path.join(out_dir, f"{output_name}.tif")

        # Salvar o raster resultante
        with rasterio.open(output_path, "w", **output_meta) as dest:
            dest.write(result_data.astype(np.float32), 1)

        print(f"Arquivo salvo em: {output_path}")

    return output_path

In [47]:

#process_rasters(dsm_path,dsm_path, "add", outdir, "dsm_p_dsm")


Arquivo salvo em: /home/ljp238/Desktop/testing_gedi/dsm_p_dsm.tif


In [ ]:
dtm_path = "/media/ljp238/12TBWolf/RSPROX/GEDI_GRID/tiles/N13E103/N13E103_GEDI03_dtm____GWR_12.tif"
egm_path = "/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N13E103/N13E103_edem_EGM.tif"
wgs_path = "/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/N13E103/N13E103_edem_W84.tif"



Arquivo salvo em: /home/ljp238/Desktop/testing_gedi/N13E103_GRID.tif


grid equations:
- grid=egm - wgs | egm=grid+wgs | wgs=egm−grid

In [ ]:

grid = process_rasters(egm_path,wgs_path, "subtract", outdir, "N13E103_GRID")
#dsm_grid = process_rasters(dsm_path,grid, "add", outdir, "N13E103_DSM_GRID")
dtm_grid = process_rasters(dtm_path,grid, "add", outdir, "N13E103_DTM_GRID")

KeyboardInterrupt: 

In [57]:
from glob import glob

In [69]:
dtm_files = glob("/media/ljp238/12TBWolf/RSPROX/GEDI_GRID/tiles/*/*_GEDI03_dtm____GWR_12.tif")
egm_files= glob("/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/*/*_edem_EGM.tif")
wgs_files = glob("/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/*/*_edem_W84.tif")
outdir_pstprox = "/home/ljp238/Desktop/testing_gedi/POSTPROCESSING_DEMS/"
assert len(dtm_files) == len(egm_files) == len(wgs_files), 'Missing files'

In [70]:
for i in range(len(dtm_files)):
    dtm_path = dtm_files[i]
    egm_path = egm_files[i]
    wgs_path = wgs_files[i]
    tilename = os.path.dirname(dtm_path).split('/')[-1]
    print(tilename)
    # grid_path = process_rasters(egm_path,wgs_path, "subtract", outdir_pstprox, f"{tilename}_GRID")
    # dtm_grid_path = process_rasters(dtm_path,grid_path, "add", outdir_pstprox, f"{tilename}_DTM_GRID")

N09E105
N09E106
N10E104
N10E105
N10E106
N11E104
N11E105
N12E103
N12E104
N12E105
N13E103
N13E104
N13E105
S01W063
S01W064
S02W063
S02W064


In [ ]:
import os
import concurrent.futures

def process_tile(i):
    dtm_path = dtm_files[i]
    egm_path = egm_files[i]
    wgs_path = wgs_files[i]
    tilename = os.path.dirname(dtm_path).split('/')[-1    
    print(f"Processando {tilename}...")

    # Executa as operações
    grid_path = process_rasters(egm_path, wgs_path, "subtract", outdir_pstprox, f"{tilename}_GRID")
    dtm_grid_path = process_rasters(dtm_path, grid_path, "add", outdir_pstprox, f"{tilename}_DTM_GRID")

    return tilename  # Apenas para indicar conclusão

# Número de threads (ajuste conforme necessário)
num_threads = min(4, len(dtm_files))  

# Executa em paralelo
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    results = list(executor.map(process_tile, range(len(dtm_files))))

print("Processamento concluído!")


In [1]:
import os

In [2]:
os.getcwd()

'/home/ljp238/Documents/UoE/libb_prox'

In [ ]:
files = glob('/home/ljp238/Desktop/testing_gedi/POSTPROCESSING_DEMS/*')

In [1]:
import os
os.system('notify-send "Python Script Finished" "Your script has completed execution."')


0

In [3]:
import os

def notify_send(title: str, message: str, duration: int = 5):
    """
    Displays a notification on Linux using notify-send.
    
    Parameters:
    title (str): The notification title.
    message (str): The notification message.
    duration (int): Time in seconds to display the notification.
    """
    os.system(f'notify-send -t {duration * 1000} "{title}" "{message}"')

# Example usage
notify_send("Python Script Finished", "Your script has completed execution.", 50)


In [4]:
gpass = "Exeter@2023"
from glob import glob 
from os import remove

In [12]:
pattern = "/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/*/*.parquet"
pattern = "/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/*/*_dsm_dtm_kmeans.tif"
pattern = "/media/ljp238/12TBWolf/RSPROX/OUTPUT_TILES/TILES12/*/*.tif.aux.xml"
files = glob(pattern); print(len(pattern))
files = list(set(files)); print(len(pattern))
#for fi in files:remove(fi)

66
66


In [13]:
method = 'mlinterps' 
dem_ipath = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/N13E103/N13E103_multi_DTM_LiDAR.tif"
tile_dpath = os.path.dirname(dem_ipath,'vfill')
bname = os.path.basename(dem_ipath).replace('.tif', f'_{method}.tif')
os.makedirs(tile_dpath,exist_ok=True)
dem_opath = os.path.join(tile_dpath, bname)


NameError: name 'os' is not defined

In [4]:
from glob import glob
import os

In [8]:
fpath = "/media/ljp238/12TBWolf/RSPROX/OUT_TILES/TILES12/*/*.parquet"
files = glob(fpath)
files

[]

In [6]:
for fi in files: os.remove(fi)